In [10]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from subprocess import check_output
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.optimizers import Adam # - Works
from keras.callbacks import TensorBoard
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score, recall_score, f1_score
import random
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import time
import math
import numpy as np
import collections
import sys
import os
from simulation_random_single import simulate # import the simulation file

dir_prefix = ''
def split_to_int(myStr): # the function to convert the string of classes into an array with integers
  return [int(char) for char in myStr]


In [ ]:



####################  AGNews-10pct Model / data parameters  (No Change Here)  ####################
#File Path
TRAIN_FILE_PATH = "agnews_10pct_data/train_10pct_new.csv"
TEST_FILE_PATH = "agnews_10pct_data/test_10pct_new.csv"


#Load Data
traindata = pd.read_csv(TRAIN_FILE_PATH, header=None)
testdata = pd.read_csv(TEST_FILE_PATH, header=None)
#Set Column Names 
traindata.columns = ['ClassIndex', 'Title', 'Description']
testdata.columns = ['ClassIndex', 'Title', 'Description']#Combine Title and Description
x_train = traindata['Title'] + " " + traindata['Description'] # Combine title and description (better accuracy than using them as separate features)
y_train = traindata['ClassIndex'].apply(lambda x: x-1).values # Class labels need to begin from 0
x_test = testdata['Title'] + " " + testdata['Description'] # Combine title and description (better accuracy than using them as separate features)
y_test = testdata['ClassIndex'].apply(lambda x: x-1).values # Class labels need to begin from 0
dataset = 'agnews_10pct'

#Max Length of sentences in Train Dataset
x_train = np.array(x_train)
x_test = np.array(x_test)

# #################### FashionMNIST Model / data parameters  (No Change Here)  ####################
# input_shape = (28, 28, 1)
# dataset = 'fashionMNIST'
# # the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
# # Scale images to the [0, 1] range
# x_train = x_train.astype("float32") / 255
# x_test = x_test.astype("float32") / 255
# # Make sure images have shape (28, 28, 1)
# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)
# # print("x_train shape:", x_train.shape)
# # print(x_train.shape[0], "train samples")
# # print(x_test.shape[0], "test samples")



if not os.path.exists(dir_prefix + dataset + '_result/'):
  os.makedirs(dir_prefix + dataset + '_result/')


original_num_classes = len(np.unique(y_train))



####################  Read the simulation parameters from the list of arguments  ####################
ratio = 0.5
model = 'mn'
noise_type = 'uniform'  # Only ['uniform', 'locally-concentrated'] for num_classes = 2
classesStr = '0123'
good_user = True # Only simulate perfect
percent_noise_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
frac = 0.1



classes = split_to_int(classesStr)
num_classes = len(classes)


####################  Start the Simulation  ####################
label_map = dict()     # encode the labels
for i in range(0, len(classes)):
  label_map[classes[i]] = i



noise_ratio = ratio  # can be any number between 0-1
noise_ratio_str = str(int(noise_ratio*100))
if frac == 1:
  fracStr = 'full'
else: fracStr = 'frac' + str(frac)[-1]

if num_classes == original_num_classes: classesStr = ''


print("=========THIS ROUND: ", dataset, noise_type, noise_ratio)
y_train_noisy_filepath = dir_prefix +  dataset + '_noisy_data/' + dataset + '_' + str(num_classes) + 'cls' + classesStr + '_' + fracStr + '_' + noise_type + '_' + noise_ratio_str + '.npy'

y_train_noisy = np.load(y_train_noisy_filepath)

sampled_idx_train_filepath = dir_prefix  + dataset + '_sampled_data/' + dataset + '_' + str(num_classes) + 'cls' + classesStr + '_train_' + fracStr + '_idx.npy'

sampled_idx_train = np.load(sampled_idx_train_filepath)

sampled_idx_test_filepath = dir_prefix  + dataset + '_sampled_data/' + dataset + '_' + str(num_classes) + 'cls' + classesStr + '_test_' + fracStr + '_idx.npy'

sampled_idx_test = np.load(sampled_idx_test_filepath)

x_train_sampled = x_train[sampled_idx_train]

y_train_sampled = y_train[sampled_idx_train]

x_test_sampled = x_test[sampled_idx_test]

y_test_sampled = y_test[sampled_idx_test]



# encode the clean y_train label
for i in range(0, y_train_sampled.shape[0]):
  prev_label = y_train_sampled[i]
  # print('prev_label:', prev_label)
  y_train_sampled[i] = label_map[prev_label]
  # print('cur_label:', label_map[i]])
for i in range(0, y_test_sampled.shape[0]):
  prev_label = y_test_sampled[i]
  y_test_sampled[i] = label_map[prev_label]



user_str = ''
if good_user: user_str = 'gooduser'


############### Run the simulation and save result into a csv file  ###############
exp_df, human_flip_err_df = simulate(dataset, x_train_sampled, y_train_noisy, x_test_sampled, y_test_sampled, y_train_sampled, noise_ratio, model, label_map.values(), good_user, percent_noise_list)
exp_df.to_csv(dir_prefix + dataset + '_result/' + dataset+"_"+ model + "_" + noise_type +"_" + user_str + "_random_"+ str(num_classes) + 'cls' + classesStr + '_' + fracStr + '_' + noise_type + '_' + noise_ratio_str + ".csv")





